In [ ]:
import numpy as np
import pandas as pd 
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
dataset = pd.read_csv("Housing.csv")
X = dataset.drop("price",axis=1)
y = dataset["price"].values
y = y.reshape(len(y),1)
encode_column = ['mainroad','guestroom','basement','hotwaterheating','airconditioning','prefarea','furnishingstatus']
ct = ColumnTransformer(transformers=[('cat',OneHotEncoder(),encode_column),('area', StandardScaler(), ["area"])],remainder='passthrough')
X = ct.fit_transform(X)
scx = StandardScaler()
scy = StandardScaler()
y = scy.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

model = SVR(kernel='rbf')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(r2_score(y_test, y_pred))
print(np.concatenate((y_test.reshape(len(y_test),1),y_pred.reshape(len(y_pred),1)),1))

0.6109030514271634
[[-0.09724783 -0.13568036]
 [ 0.70436917  0.70476046]
 [-0.40628243 -0.47904747]
 [ 1.15761991  0.07335807]
 [-0.97752819 -0.85691725]
 [ 0.76430315  0.75041344]
 [-0.65912891 -0.73087192]
 [-0.97752819 -0.6536321 ]
 [-0.67785828 -0.69927742]
 [ 1.71950099  1.57488952]
 [ 1.06397306  0.98348601]
 [-0.41564711 -0.33011546]
 [-0.86515198 -0.51058254]
 [-0.7902345  -0.1156275 ]
 [-0.34072964 -0.21034802]
 [-1.12736315 -1.16322705]
 [-0.19089468 -0.40894315]
 [-0.56548207 -0.20749353]
 [-0.75277576 -0.71315518]
 [-0.26581216 -0.40785278]
 [ 0.1275046   0.4836103 ]
 [ 0.32978179  0.05381409]
 [ 0.89540874 -0.25479535]
 [-1.53940927 -1.20794594]
 [ 0.72684442  0.64538753]
 [ 0.3709864  -0.24935672]
 [ 0.55453422  0.5312946 ]
 [-0.07851846  0.06181145]
 [ 0.40844514  0.19051911]
 [ 0.63319757  0.23021214]
 [-0.24708279 -0.47317912]
 [-0.60294081 -0.22972907]
 [ 0.25861018  0.74197812]
 [-0.77150513 -0.91291908]
 [-0.56922794 -0.71060139]
 [-0.00734686 -0.29661634]
 [ 2.3188

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [2]:
import numpy as np
import pandas as pd

from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Load dataset
dataset = pd.read_csv("Housing.csv")

# Features and target
X = dataset.drop("price", axis=1)
y = dataset["price"].values.reshape(-1, 1)

# Column groups
categorical_cols = [
    'mainroad',
    'guestroom',
    'basement',
    'hotwaterheating',
    'airconditioning',
    'prefarea',
    'furnishingstatus'
]

numeric_cols = [
    'area',
    'bedrooms',
    'bathrooms',
    'stories',
    'parking'
]

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numeric_cols)
    ]
)

# Model pipeline
svr_pipeline = Pipeline(
    steps=[
        ('preprocessing', preprocessor),
        ('svr', SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1))
    ]
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# Scale target (SVR needs this)
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train).ravel()
y_test_scaled = y_scaler.transform(y_test).ravel()

# Train
svr_pipeline.fit(X_train, y_train_scaled)

# Predict (scaled)
y_pred_scaled = svr_pipeline.predict(X_test)

# Inverse transform predictions
y_pred = y_scaler.inverse_transform(
    y_pred_scaled.reshape(-1, 1)
)

# Evaluation
# Some sklearn versions don't accept the 'squared' kwarg in mean_squared_error.
mse = mean_squared_error(y_test.ravel(), y_pred.ravel())
rmse = np.sqrt(mse)
r2 = r2_score(y_test.ravel(), y_pred.ravel())

print("RMSE:", rmse)
print("R2 Score:", r2)

# Compare actual vs predicted
print(
    np.concatenate(
        (y_test.reshape(-1, 1), y_pred),
        axis=1
    )
)


RMSE: 1440215.2867547572
R2 Score: 0.28016415313533904
[[ 4585000.          4683950.25077768]
 [ 6083000.          4435137.30696852]
 [ 4007500.          2831229.71627208]
 [ 6930000.          6179230.35989203]
 [ 2940000.          2717572.0233166 ]
 [ 6195000.          6509484.07872168]
 [ 3535000.          3903582.27310335]
 [ 2940000.          3019003.79758067]
 [ 3500000.          2958705.0401163 ]
 [ 7980000.         11540855.08540652]
 [ 6755000.          5219975.78406059]
 [ 3990000.          3382570.79708578]
 [ 3150000.          3799260.22253984]
 [ 3290000.          5326911.63852993]
 [ 4130000.          6072751.40929752]
 [ 2660000.          2453965.09217201]
 [ 4410000.          4298903.11255615]
 [ 3710000.          4515946.58016278]
 [ 3360000.          3445781.54252489]
 [ 4270000.          5284396.05354485]
 [ 5005000.          7015126.56558749]
 [ 5383000.          6224436.63220493]
 [ 6440000.          4138906.67008999]
 [ 1890000.          1941936.33161019]
 [ 612500